In [13]:
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.auto import tqdm

def read_data():
    query = f"""
                SELECT domain, entity_id, state, last_changed, attributes
                FROM states
                WHERE
                    entity_id IS 'sensor.voltmeter'
                ORDER BY last_changed DESC
                """

    return (
        pd.read_sql_query(query, 'sqlite:////config/home-assistant_v2.db')
        .assign(last_changed=lambda x: pd.to_datetime(x['last_changed']))
        .sort_values('last_changed')
        .replace({'state': {'unavailable': np.nan, 'unknown': np.nan}})
        .astype({'state': float})
        .set_index('last_changed')
        ['state']
    )

In [25]:
input_experiments = [
    {'sample_rate': .001, 'moving median': 1,      'begin_time': pd.Timestamp('2021-09-13 12:06'), 'end_time': pd.Timestamp('2021-09-13 12:22')},
    {'sample_rate': .001, 'moving median': 10,     'begin_time': pd.Timestamp('2021-09-13 12:28'), 'end_time': pd.Timestamp('2021-09-13 12:45')},
    {'sample_rate': .001, 'moving median': 100,    'begin_time': pd.Timestamp('2021-09-13 12:48'), 'end_time': pd.Timestamp('2021-09-13 13:03')}, 
#     {'sample_rate': .001, 'moving median': 1_000,  'begin_time': pd.Timestamp('2021-09-13 12:06'), 'end_time': pd.Timestamp('2021-09-13 12:22')},
#     {'sample_rate': .001, 'moving median': 10_000, 'begin_time': pd.Timestamp('2021-09-13 12:24'), 'end_time': pd.Timestamp('2021-09-13 12:39')},
#     {'sample_rate': .1,   'moving median': 1,      'begin_time': pd.Timestamp('2021-09-13 12:06'), 'end_time': pd.Timestamp('2021-09-13 12:22')},
#     {'sample_rate': .1,   'moving median': 10,     'begin_time': pd.Timestamp('2021-09-13 12:24'), 'end_time': pd.Timestamp('2021-09-13 12:39')},
#     {'sample_rate': .1,   'moving median': 100,    'begin_time': pd.Timestamp('2021-09-13 12:24'), 'end_time': pd.Timestamp('2021-09-13 12:39')}, 
#     {'sample_rate': 1,    'moving median': 1,      'begin_time': pd.Timestamp('2021-09-13 12:06'), 'end_time': pd.Timestamp('2021-09-13 12:22')},
#     {'sample_rate': 1,    'moving median': 10,     'begin_time': pd.Timestamp('2021-09-13 12:24'), 'end_time': pd.Timestamp('2021-09-13 12:39')},
]

In [16]:
df = read_data()

In [26]:
output_experiments = []
for experiment in tqdm(input_experiments):
    df_selected = df.loc[lambda x: (experiment['begin_time'] < x.index) & (experiment['end_time'] > x.index)]
    output_experiments.append({'sample_rate': experiment['sample_rate'], 'moving median': experiment['moving median'], 'rstd': df_selected.std()/df_selected.median(), 'med': df_selected.median()})

  0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
pd.DataFrame(output_experiments)

,sample_rate,moving median,rstd,med
0,0.001,1,0.039947,1.03
1,0.001,10,0.026007,1.03
